<a href="https://colab.research.google.com/github/patty-13/DEEP_LEARNING_NJIT/blob/main/HW5_DEEPLEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download data here: https://drive.google.com/file/d/169ghKW0xyLbOuMa7rrRUj6XN3QrA-FI0/view?usp=share_link. Copy it to your own drive and get the file id to fill the following cell.


In [ ]:
#!gdown --id 'your own file id' --output data.zip
#!gdown --id '1u3qSB0ECgtz98MuWiQOtiJrhgodiHMAC' --output data.zip
#!unzip data.zip
from google.colab import drive
drive.mount('/content/gdrive')
!unzip '/content/gdrive/MyDrive/data.zip'

Streaming output truncated to the last 5000 lines.
  inflating: data/train/uttr16611.pt  
  inflating: __MACOSX/data/train/._uttr16611.pt  
  inflating: data/train/uttr34368.pt  
  inflating: __MACOSX/data/train/._uttr34368.pt  
  inflating: data/train/uttr23681.pt  
  inflating: __MACOSX/data/train/._uttr23681.pt  
  inflating: data/train/uttr23510.pt  
  inflating: __MACOSX/data/train/._uttr23510.pt  
  inflating: data/train/uttr32664.pt  
  inflating: __MACOSX/data/train/._uttr32664.pt  
  inflating: data/train/uttr45094.pt  
  inflating: __MACOSX/data/train/._uttr45094.pt  
  inflating: data/train/uttr45305.pt  
  inflating: __MACOSX/data/train/._uttr45305.pt  
  inflating: data/train/uttr53912.pt  
  inflating: __MACOSX/data/train/._uttr53912.pt  
  inflating: data/train/uttr54071.pt  
  inflating: __MACOSX/data/train/._uttr54071.pt  
  inflating: data/train/uttr28367.pt  
  inflating: __MACOSX/data/train/._uttr28367.pt  
  inflating: data/train/uttr39013.pt  
  inflating: __MACOS

## Dataset

- 600 speakers, preprocess the raw waveforms into mel-spectrograms.


For efficiency, we segment the mel-spectrograms into segments in the training step.

In [ ]:
!pip install conformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/Masao-Someki/Conformer.git

Cloning into 'Conformer'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 87 (delta 37), reused 65 (delta 24), pack-reused 0
Receiving objects: 100% (87/87), 72.83 KiB | 4.85 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [ ]:
!pip install torchmetrics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 10.0 MB/s eta 0:00:00


In [ ]:
%cd '/content/Conformer/'

/content/Conformer


In [ ]:
import torch
import random
import numpy as np
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed = 1213
same_seeds(seed)

In [ ]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import sys
sys.path.append('/content/Conformer')

class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=150):
    self.data_dir = data_dir
    self.segment_len = segment_len

    # Get the total number of speaker.
    self.data = pd.read_csv(os.path.join(data_dir,'train_label.csv'))
    self.speaker_num = len(self.data.speaker_id.unique())

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path, speaker = self.data.iloc[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, 'train', feat_path))

    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker

  def get_speaker_number(self):
    return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#from conformer import ConformerBlock
#from conformer import Conformer
from CF import Conformer

class Classifier(nn.Module):
  def __init__(self, d_model=256, n_spks=600, dropout=0.2):
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    self.prenet = nn.Linear(40, d_model)
    # TODO:
    #  1. Use multiple Transformer Encoder layers: nn.TransformerEncoder()
    #  2. Change Transformer to Conformer. https://arxiv.org/abs/2005.08100
    self.conformer_block = Conformer(
        # dim=d_model,
        # depth=2,
        # dim_head=64,
        # heads=8,  # set 1
        # ff_mult=4,
        # conv_expansion_factor=2,
        # conv_kernel_size=31,
        # attn_dropout=dropout,
        # ff_dropout=dropout,
        # conv_dropout=dropout
        d_model=d_model,
        ff1_hsize=2048,
        ff1_dropout=dropout,
        n_head=16,
        mha_dropout=dropout,
        kernel_size=31,
        conv_dropout=dropout,
        ff2_hsize=2048,
        ff2_dropout=dropout,
    )

    # self.encoder_layer = nn.TransformerEncoderLayer(
    #   d_model=d_model,
    #   dim_feedforward=1024,
    #   nhead=64
    # )
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)

    # Project the the dimension of features from d_model into speaker nums.
    self.pred_layer = nn.Sequential(
      # nn.Linear(d_model, 2*d_model),
      # nn.BatchNorm1d(2 * d_model),
      # nn.ReLU(),
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model)
    out = self.prenet(mels)
    # out: (length, batch size, d_model)
    #out = out.permute(1, 0, 2)
    # The encoder layer expect features in the shape of (length, batch size, d_model).
    out = self.conformer_block(out)
    #out = self.encoder_layer(out)
    # out: (batch size, length, d_model)
    #out = out.transpose(0, 1)
    # mean pooling
    stats = out.mean(dim=1)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [ ]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


# Model Function
- Model forward function.

In [ ]:
import torch


def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [ ]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)


In [ ]:
class AMSoftmaxLoss(nn.Module):
    def __init__(self, in_features=256, out_features=600, m=0.35, s=30.0):
        super(AMSoftmaxLoss, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.m = m
        self.s = s
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, target):
        # Normalize input features
        x = F.normalize(input, p=2, dim=1)
        # Normalize weight vectors
        w = F.normalize(self.weight, p=2, dim=1)

        # Cosine similarity between x and w
        cos_theta = F.linear(x, w.T)

        # Add margin to the cosine similarity
        cos_theta_m = cos_theta - self.m

        # Compute the AM-Softmax loss
        loss = F.cross_entropy(self.s * cos_theta_m, target)

        return loss


In [ ]:
class AMSoftmax(nn.Module):
  def __init__(self, in_features=256, n_classes=600, s=30, m=0.4):
    super(AMSoftmax,self).__init__()
    self.linear = nn.Linear(in_features, n_classes, bias=False)
    self.s = s
    self.m = m

  def forward(self, *inputs):
    x_vector = F.normalize(inputs[0], p=2, dim=-1)
    self.linear.weight.data = F.normalize(self.linear.weight.data, p =2, dim=-1)
    logits = self.linear(x_vector)
    scaled_logits = (logits -self.m)*self.s
    return scaled_logits - self.am_logsumexp(logits)

  def am_logsumexp(self, logits):
    max_x = torch.max(logits, dim=-1)[0].unsqueeze(-1)
    term1 = (self.s*(logits - (max_x + self.m))).exp()
    term2 = (self.s * (logits - max_x)).exp().sum(-1).unsqueez(-1) - (self.s * (logits - max_x)).exp()
    return self.s*max_x + (term2 + term1).log()


# Main function

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    "data_dir": "./data",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 150000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(n_spks=speaker_num).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [02:08<00:00, 15.58 step/s, accuracy=0.28, loss=3.19, step=2000]
Valid: 100% 6240/6250 [00:14<00:00, 429.08 uttr/s, accuracy=0.41, loss=2.63]
Train: 100% 2000/2000 [02:00<00:00, 16.60 step/s, accuracy=0.66, loss=1.49, step=4000]
Valid: 100% 6240/6250 [00:15<00:00, 409.31 uttr/s, accuracy=0.58, loss=1.82]
Train: 100% 2000/2000 [01:59<00:00, 16.79 step/s, accuracy=0.56, loss=1.80, step=6000]
Valid: 100% 6240/6250 [00:15<00:00, 415.20 uttr/s, accuracy=0.66, loss=1.46]
Train: 100% 2000/2000 [01:58<00:00, 16.83 step/s, accuracy=0.75, loss=1.00, step=8000]
Valid: 100% 6240/6250 [00:15<00:00, 403.28 uttr/s, accuracy=0.71, loss=1.25]
Train: 100% 2000/2000 [01:59<00:00, 16.79 step/s, accuracy=0.78, loss=1.08, step=1e+4]
Valid: 100% 6240/6250 [00:15<00:00, 412.36 uttr/s, accuracy=0.73, loss=1.15]
Train:   0% 4/2000 [00:00<02:05, 15.93 step/s, accuracy=0.78, loss=0.74, step=1e+4]

Step 10000, best model saved. (accuracy=0.7319)


...

Step 110000, best model saved. (accuracy=0.9261)


Train: 100% 2000/2000 [02:00<00:00, 16.57 step/s, accuracy=1.00, loss=0.00, step=112000]
Valid: 100% 6240/6250 [00:15<00:00, 404.88 uttr/s, accuracy=0.92, loss=0.43]
Train: 100% 2000/2000 [02:01<00:00, 16.44 step/s, accuracy=1.00, loss=0.00, step=114000]
Valid: 100% 6240/6250 [00:15<00:00, 415.34 uttr/s, accuracy=0.92, loss=0.42]
Train: 100% 2000/2000 [02:03<00:00, 16.25 step/s, accuracy=1.00, loss=0.00, step=116000]
Valid: 100% 6240/6250 [00:14<00:00, 441.12 uttr/s, accuracy=0.92, loss=0.42]
Train: 100% 2000/2000 [01:57<00:00, 16.95 step/s, accuracy=1.00, loss=0.00, step=118000]
Valid: 100% 6240/6250 [00:14<00:00, 435.71 uttr/s, accuracy=0.92, loss=0.41]
Train: 100% 2000/2000 [02:00<00:00, 16.66 step/s, accuracy=1.00, loss=0.00, step=120000]
Valid: 100% 6240/6250 [00:15<00:00, 413.95 uttr/s, accuracy=0.92, loss=0.42]
Train:   0% 4/2000 [00:00<01:55, 17.23 step/s, accuracy=1.00, loss=0.00, step=120004]

Step 120000, best model saved. (accuracy=0.9261)


Train: 100% 2000/2000 [01:59<00:00, 16.74 step/s, accuracy=1.00, loss=0.01, step=122000]
Valid: 100% 6240/6250 [00:15<00:00, 415.95 uttr/s, accuracy=0.92, loss=0.38]
Train: 100% 2000/2000 [01:58<00:00, 16.87 step/s, accuracy=1.00, loss=0.00, step=124000]
Valid: 100% 6240/6250 [00:15<00:00, 410.18 uttr/s, accuracy=0.92, loss=0.39]
Train: 100% 2000/2000 [02:00<00:00, 16.60 step/s, accuracy=1.00, loss=0.00, step=126000]
Valid: 100% 6240/6250 [00:15<00:00, 404.39 uttr/s, accuracy=0.92, loss=0.40]
Train:  23% 467/2000 [00:30<01:28, 17.30 step/s, accuracy=1.00, loss=0.00, step=126467]

KeyboardInterrupt: ignored

In [ ]:
%cd '/content/Conformer'

/content/Conformer


# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    self.data_dir = data_dir
    self.data = pd.read_csv(os.path.join(data_dir,'test_id.csv'))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path = self.data.feat_path[index]
    mel = torch.load(os.path.join(self.data_dir, 'test', feat_path))

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)


## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "./data",
    "model_path": "./model.ckpt",
    "output_path": "./output_conformer_4_9325.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = 600
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, pred])

  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!
[Info]: Finish loading data!


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[Info]: Finish creating model!


  0%|          | 0/6944 [00:00<?, ?it/s]